In [ ]:
### Inputs: traces_dict, node_details_dict and trace_details_dict
# Node details dict= nid: [nis, type]
### Config file: DB split and SLtype split
### Outputs: updated_node_details

In [1]:
import pickle
import yaml
import random

In [5]:
def pkl_to_dict(file_path):
    with open(file_path, 'rb') as pkl_file:
        T_prime = pickle.load(pkl_file)
    return T_prime

def read_yaml(file):
    with open(file, 'r') as f:
        data = yaml.safe_load(f)
    return data


In [6]:
# Read in configs
config = read_yaml('enrichment_config.yaml')
databases = config['Databases']
sl_types = config['SLTypeSplit']

db_split_arr = [[db_name, info['percentage']] for db_name, info in databases.items()]# [[DB1, 30%],...]
sl_type_split = [[sl_type, info['percentage']] for sl_type, info in sl_types.items()]# [[Relay, 30%],...]

# Node details dict= nid: [nis, SF, DB_name] (or) [nis, SL, SL_type]
node_dets = pkl_to_dict('node_details_data.pkl')
sf_arr = [nid for nid, n_info in node_dets.items() if n_info[1] == "db"]
sl_arr = [nid for nid, n_info in node_dets.items() if n_info[1] != "db"]

sf_count = len(sf_arr)
sl_count = len(sl_arr)
total_nodes = sf_count + sl_count

def percent_to_count(arr, count):
    for idx, i in enumerate(arr):
        name = i[0]
        arr[idx] = [name,int(count * (i[1])/100)]
    return arr

db_split_arr = percent_to_count(db_split_arr, sf_count)
sl_type_split = percent_to_count(sl_type_split, sl_count)

print(len(sf_arr))
print(len(sl_arr))
print(db_split_arr)
print(sl_type_split)

def assign_nodes_to_types(arr, sfsl_arr):
    # Assign nodes to db and sl types
    for i in arr:
        name = i[0]
        for _ in range(i[1]):
            nid = sfsl_arr.pop(random.randint(0, len(sfsl_arr) - 1))
            node_dets[nid].append(name)
    return node_dets
node_dets = assign_nodes_to_types(db_split_arr, sf_arr)
node_dets = assign_nodes_to_types(sl_type_split, sl_arr)

2330
1652
[['MongoDB', 699], ['Redis', 699], ['Postgres', 932]]
[['Relay', 495], ['High', 330], ['Low', 826]]


In [7]:
check = []
for i in node_dets.values():
    if len(i) != 3:
        check.append(i)
print(len(check))
print(len(node_dets))

1
3982


In [2]:
'''
Object id Enrichment
'''

class Wl_config:
    def __init__(self, record_count, record_size_dist,\
                 data_access_pattern, rw_ratio, async_sync_ratio):
        self.record_count = record_count
        self.record_size_dist = record_size_dist
        self.data_access_pattern = data_access_pattern
        self.rw_ratio = rw_ratio
        self.async_sync_ratio = async_sync_ratio

wl1 = Wl_config(100, 'uniform', 'zipfian', 0.5, 0.1) # to be read from config file

# convert edges_list to node_calls_dict format
def convert_to_node_calls_dict(edges_list):
    node_calls_dict = {}
    for edge in edges_list:
        if edge[0] not in node_calls_dict:
            node_calls_dict[edge[0]] = []
        node_calls_dict[edge[0]].append(edge[1])
    return node_calls_dict

# for each trace in traces_dict
    # extract sf nodes in trace
    # for each sf node in trace, perform enrichment 
        # compute in-degree of sf node
        # generate indeg num of data ops



'\nObject id Enrichment\n'